<a href="https://colab.research.google.com/github/francisconetodata/Jupyter_Notebook_Exemplos/blob/main/feedback_prize_effectiveness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [ ]:
%%capture
from google.colab import files
files.upload()

In [3]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


rm: cannot remove '/root/.kaggle': No such file or directory


In [6]:
nome = !kaggle competitions download -c feedback-prize-effectiveness

In [7]:
import zipfile
import time
time.sleep(5)
with zipfile.ZipFile(f"/content/{(nome.fields()[0][0]).replace(':','')}","r") as zip_ref:
    zip_ref.extractall("")

In [9]:
import pandas as pd

In [10]:
df_train = pd.read_csv('train.csv')
df_train

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate
...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective


In [16]:
df_train.info()
df_train = df_train.drop(columns=['discourse_id','essay_id'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36765 entries, 0 to 36764
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   discourse_id             36765 non-null  object
 1   essay_id                 36765 non-null  object
 2   discourse_text           36765 non-null  object
 3   discourse_type           36765 non-null  object
 4   discourse_effectiveness  36765 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB


In [12]:
!pip install h2o
import h2o
from h2o.automl import H2OAutoML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.0 MB 48 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.2-py2.py3-none-any.whl size=177070945 sha256=b3e5025080ad9c0ad6c68ad125559a809464972b9c924bc8167f1e5692bec88d
  Stored in directory: /root/.cache/pip/wheels/a5/2b/ae/534595df09001e1573b1046be62a3f049507307340119db65c
Successfully built h2o


In [13]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp7u4mo47y
  JVM stdout: /tmp/tmp7u4mo47y/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp7u4mo47y/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_unknownUser_mhy2oq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [14]:
from h2o.estimators import H2OWord2vecEstimator, H2OGradientBoostingEstimator

In [17]:
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can",
              "lines","re","what","there","all","we","one","the",
              "a","an","of","or","in","for","by","on","but","is",
              "in","a","not","with","as","was","if","they","are",
              "this","and","it","have","from","at","my","be","by",
              "not","that","to","from","com","org","like","likes",
              "so"]
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words
def predict(job_title,w2v, gbm):
    words = tokenize(h2o.H2OFrame(job_title).ascharacter())
    job_title_vec = w2v.transform(words, aggregate_method="AVERAGE")
    print(gbm.predict(test_data=job_title_vec))
job_titles = h2o.H2OFrame(df_train)
words = tokenize(job_titles["discourse_text"])
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model.train(training_frame=words)
job_title_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
valid_job_titles = ~ job_title_vecs["C1"].isna()
data = job_titles[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])
data_split = data.split_frame(ratios=[0.8])
gbm_model = H2OGradientBoostingEstimator()
gbm_model.train(x = job_title_vecs.names,
                y="discourse_effectiveness",
                training_frame = data_split[0],
                validation_frame = data_split[1])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
word2vec Model Build progress: |█████████████████████████████████████████████████| (done) 100%
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1655844999810_3


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,150.0,65256.0,5.0,5.0,5.0,23.0,32.0,29.946667




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.254549875690799
RMSE: 0.5045293605835036
LogLoss: 0.7234312371337526
Mean Per-Class Error: 0.4760857221247905
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,Adequate,Effective,Ineffective,Error,Rate
0,15275.0,1298.0,120.0,0.084946,"1,418 / 16,693"
1,3674.0,3754.0,12.0,0.495430,"3,686 / 7,440"
2,4208.0,134.0,779.0,0.847881,"4,342 / 5,121"
3,23157.0,5186.0,911.0,0.322896,"9,446 / 29,254"



Top-3 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc,validation_rmse,validation_logloss,validation_classification_error,validation_auc,validation_pr_auc
0,,2022-06-21 21:11:05,0.038 sec,0.0,0.666667,1.098612,0.579716,NaN,NaN,0.666667,1.098612,0.582911,NaN,NaN
1,,2022-06-21 21:11:08,2.999 sec,1.0,0.651510,1.054943,0.385076,NaN,NaN,0.652447,1.057643,0.395495,NaN,NaN
2,,2022-06-21 21:11:12,7.064 sec,5.0,0.606937,0.942740,0.376974,NaN,NaN,0.610925,0.953382,0.384031,NaN,NaN
3,,2022-06-21 21:11:16,11.112 sec,12.0,0.565258,0.852440,0.369967,NaN,NaN,0.573739,0.874781,0.379099,NaN,NaN
4,,2022-06-21 21:11:20,15.495 sec,20.0,0.541789,0.804365,0.359746,NaN,NaN,0.555177,0.840084,0.376033,NaN,NaN
5,,2022-06-21 21:11:25,19.523 sec,27.0,0.529194,0.778001,0.349867,NaN,NaN,0.546851,0.825512,0.373234,NaN,NaN
6,,2022-06-21 21:11:29,23.589 sec,35.0,0.518700,0.755311,0.339646,NaN,NaN,0.541266,0.816001,0.370035,NaN,NaN
7,,2022-06-21 21:11:33,27.852 sec,40.0,0.513618,0.743970,0.333288,NaN,NaN,0.538818,0.811795,0.368435,NaN,NaN
8,,2022-06-21 21:11:37,32.097 sec,45.0,0.509024,0.733686,0.328229,NaN,NaN,0.536946,0.808770,0.367502,NaN,NaN
9,,2022-06-21 21:11:40,34.691 sec,50.0,0.504529,0.723431,0.322896,NaN,NaN,0.535462,0.806410,0.366836,NaN,NaN



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,C28,2658.524902,1.000000,0.122962
1,C94,1822.400879,0.685493,0.084290
2,C53,1220.233521,0.458989,0.056438
3,C64,750.737244,0.282389,0.034723
4,C82,703.904785,0.264773,0.032557
5,C34,685.005981,0.257664,0.031683
6,C13,386.597656,0.145418,0.017881
7,C52,383.704285,0.144330,0.017747
8,C61,382.792725,0.143987,0.017705
9,C95,364.713257,0.137186,0.016869



See the whole table with table.as_data_frame()


In [23]:
df_teste = pd.read_csv('test.csv')
df_teste_h2o = h2o.H2OFrame(df_teste)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [25]:
words = tokenize(df_teste_h2o["discourse_text"])
words

C1
making
choices
life
very
difficult
people
often
ask
advice
when


In [26]:
job_title_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [28]:
valid_job_titles = ~ job_title_vecs["C1"].isna()
data = df_teste_h2o[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])
data

discourse_id,essay_id,discourse_text,discourse_type,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself.,Lead,-0.116962,-0.0812548,0.0998818,0.0593083,0.180247,-0.113838,0.110334,-0.0357309,-0.231736,0.0970907,0.260505,0.0305562,0.168448,-0.0879192,-0.101235,0.274688,-0.120317,-0.0204031,0.0838031,-0.102465,-0.129602,-0.134877,0.0363406,0.0340384,0.0681239,-0.148496,0.179834,0.0175271,0.200933,0.0869992,0.0225082,-0.140245,-0.0202655,-0.0236603,-0.0480326,-0.0153716,-0.063278,-0.196962,-0.125917,0.0248108,-0.0435777,-0.0962189,0.0955799,-0.129947,-0.151795,0.00455975,-0.00341906,-0.051158,0.0120851,0.0138677,0.0783581,-0.0396061,-0.0699766,-0.296287,-0.0707017,-0.0150263,0.073767,-0.07109,0.0711481,0.0304143,-0.0675578,-0.0637406,-0.104465,-0.0464773,-0.0406128,-0.188779,0.140653,-0.204081,-0.00760493,0.0371671,0.209544,0.0216773,0.104881,-0.129204,-0.0711279,0.182698,-0.0848085,0.23903,0.106281,0.0402187,0.0600738,0.0382501,-0.232116,0.0857242,0.169258,-0.0993967,0.0438,-0.366312,0.0473257,-0.139301,-0.10062,0.0046995,-0.0937067,0.0313055,-0.11886,-0.0570143,-0.111319,-0.0408519,-0.00761787,0.138388
5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person make a better choice,Position,-0.0117604,-0.110375,0.308904,-0.0336143,0.306382,-0.120587,-0.0095482,-0.0983246,-0.0831073,0.186336,0.310013,-0.029228,0.220728,-0.145403,-0.143997,0.39707,-0.0762987,-0.0143508,-0.0497169,-0.221584,-0.264887,-0.134671,-0.037646,0.0541616,0.000419765,-0.195878,0.234296,0.024258,0.213353,0.134526,0.124339,-0.178302,0.023389,0.0477233,-0.0345343,-0.0189974,-0.0920509,-0.377399,-0.0263919,0.076961,0.0237785,-0.301568,0.00993782,-0.258646,-0.177694,0.0517221,-0.134129,-0.180827,0.123442,0.00728754,0.0798115,0.0112907,-0.01053,-0.49577,0.019328,-0.187625,-0.00460631,-0.18497,0.0426602,0.100325,-0.098557,0.0771792,-0.124963,-0.0439819,-0.0408186,-0.100382,0.298101,-0.368885,-0.0999208,0.0520494,0.169651,0.165685,0.179793,-0.325612,-0.106456,0.31906,-0.222358,0.218318,0.0155535,0.216769,-0.10619,-0.0373523,-0.19938,0.206701,0.169348,-0.0259807,-0.0501884,-0.432579,0.090239,-0.202479,-0.0356052,0.0955891,-0.0835985,-0.0655804,-0.233404,0.0186983,-0.0596854,-0.112096,0.0277749,0.0418349
9790d835736b,D72CB1C11673,it can decrease stress levels,Claim,0.288598,0.0949016,0.411633,-0.332298,0.36922,-0.339857,0.0746542,-0.0661681,-0.582212,-0.547861,-0.17564,-0.0735762,-0.0398702,-0.332311,-0.0184139,0.0150756,0.234479,0.00178583,0.427855,0.294366,-0.240329,-0.0205247,-0.280366,0.290076,-0.158671,0.029584,-0.145025,0.360733,0.0367388,0.173083,-0.354571,0.175742,-0.0268756,-0.537239,0.0785585,-0.240299,-0.188896,-0.134675,-0.292142,0.150973,-0.0205816,-0.139249,0.0337636,0.416262,-0.246724,0.00923895,-0.116383,-0.0185247,0.0162372,0.0901339,0.737934,0.2543,-0.0296082,-0.249415,-0.172238,0.0285428,-0.00939504,-0.256415,-0.210372,-0.208469,0.0829358,-0.215433,-0.0974961,-0.391373,-0.2776,0.167165,0.091476,0.0850647,0.265502,-0.304303,-0.01717,0.064652,0.249805,0.236379,0.214775,0.199022,0.119845,-0.0911796,-0.0348228,-0.0343797,0.0719288,-0.0818688,-0.288161,-0.230704,0.0541875,0.283799,0.00511093,-0.220599,-0.0391753,-0.120631,-0.363755,0.0336206,-0.020258,0.21253,-0.205827,-0.0141678,-0.117673,-0.119434,0.0601992,0.339277
75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim,0.0846864,-0.0847975,0.115874,

In [31]:
y_pred = gbm_model.predict(data)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [39]:
df_pred = y_pred.as_data_frame().merge(df_teste['discourse_id'], how= 'inner',left_index = True, right_index = True)

In [42]:
df_pred = df_pred[['discourse_id','Ineffective','Adequate','Effective']]
df_pred.to_csv('pred.csv')

In [43]:
df_pred

,discourse_id,Ineffective,Adequate,Effective
0,a261b6e14276,0.060097,0.290824,0.649079
1,5a88900e7dc1,0.057430,0.662988,0.279582
2,9790d835736b,0.070179,0.526576,0.403245
3,75ce6d68b67b,0.091804,0.593476,0.314720
4,93578d946723,0.066269,0.677905,0.255826
5,2e214524dbe3,0.043279,0.222297,0.734424
6,84812fc2ab9f,0.049782,0.267658,0.682560
7,c668ff840720,0.074104,0.531648,0.394248
8,739a6d00f44a,0.099132,0.463583,0.437285
9,bcfae2c9a244,0.034796,0.353869,0.611334
